In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
from pytorch_lightning import seed_everything

from srai.embedders import GTFS2VecEmbedder
from srai.constants import REGIONS_INDEX

# Example on artificial data

## Define features and regions

In [ ]:
features_gdf = gpd.GeoDataFrame(
    {
        "trip_count_at_6": [1, 0, 0],
        "trip_count_at_7": [1, 1, 0],
        "trip_count_at_8": [0, 0, 1],
        "directions_at_6": [
            {"A", "A1"},
            {"B", "B1"},
            {"C"},
        ],
    },
    geometry=gpd.points_from_xy([1, 2, 5], [1, 2, 2]),
    index=pd.Index(name="stop_id", data=[1, 2, 3]),
)
features_gdf

In [ ]:
regions_gdf = gpd.GeoDataFrame(
    geometry=[
        Polygon([(0, 0), (0, 3), (3, 3), (3, 0)]),
        Polygon([(4, 0), (4, 3), (7, 3), (7, 0)]),
        Polygon([(8, 0), (8, 3), (11, 3), (11, 0)]),
    ],
    index=pd.Index(name=REGIONS_INDEX, data=["ff1", "ff2", "ff3"]),
)
regions_gdf

In [ ]:
ax = regions_gdf.plot()
features_gdf.plot(ax=ax, color="red")

In [ ]:
joint_gdf = gpd.GeoDataFrame()
joint_gdf.index = pd.MultiIndex.from_tuples(
    [("ff1", 1), ("ff1", 2), ("ff2", 3)],
    names=[REGIONS_INDEX, "stop_id"],
)
joint_gdf

## Get features without embedding them

In [ ]:
embedder = GTFS2VecEmbedder(skip_autoencoder=True)
res = embedder.transform(regions_gdf, features_gdf, joint_gdf)
res

## Fit and train the embedder

In [ ]:
seed_everything(42)
embedder = GTFS2VecEmbedder(hidden_size=2, embedding_size=4)
embedder.fit(regions_gdf, features_gdf, joint_gdf)
res = embedder.transform(regions_gdf, features_gdf, joint_gdf)
res